<a href="https://colab.research.google.com/github/rjatelo30/Machine-learning-2021/blob/main/Task_1_House_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from keras.layers import Dense
import matplotlib
import matplotlib.pyplot as plt
from keras.optimizers import Adam

In [83]:
uploaded = files.upload()

**Import Dataset**

In [110]:
data = pd.read_csv('/content/Housing Pricing.csv')
data.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500


In [85]:
data.drop('Id', inplace = True, axis = 1)

In [86]:
og_features = list(data.columns)
len(og_features)

76

**Check for missing values**

In [87]:
features_valmiss = data.columns[data.isna().any()]

len(features_valmiss)

15

In [88]:
bfill, mean_fill = [], []

**Check the percentage of missing values **

In [89]:
for feature in features_valmiss:
    print(feature,'', data[feature].isna().sum()/len(data)*100, '', data[feature].dtype)
    if data[feature].dtype == 'float64':
      mean_fill.append(feature)
    else:
        bfill.append(feature)

LotFrontage  17.73972602739726  float64
MasVnrType  0.547945205479452  object
MasVnrArea  0.547945205479452  float64
BsmtQual  2.5342465753424657  object
BsmtCond  2.5342465753424657  object
BsmtExposure  2.6027397260273974  object
BsmtFinType1  2.5342465753424657  object
BsmtFinType2  2.6027397260273974  object
Electrical  0.0684931506849315  object
FireplaceQu  47.26027397260274  object
GarageType  5.5479452054794525  object
GarageYrBlt  5.5479452054794525  float64
GarageFinish  5.5479452054794525  object
GarageQual  5.5479452054794525  object
GarageCond  5.5479452054794525  object


Fill the lists respectively with the method used as in the code above

In [90]:
bfill

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [91]:
mean_fill

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [92]:
for feature in mean_fill:
  data[feature].fillna(data[feature].mean(), inplace = True)

In [93]:
for feature in bfill:
  data[feature].fillna(method = 'bfill', inplace = True)

**Confirm that the empty cells have been filled**

In [94]:
data.columns[data.isna().any()]

Index(['FireplaceQu'], dtype='object')

In [95]:
data['FireplaceQu'].fillna(method = 'ffill', inplace = True)

**Encode the dataset**

In [96]:
cat_features = list(data.select_dtypes(include=['object']).columns)
cat_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [97]:
numerical = list(set(og_features) - set(cat_features))
numerical

['EnclosedPorch',
 'MasVnrArea',
 'OpenPorchSF',
 'ScreenPorch',
 'MoSold',
 'YrSold',
 'SalePrice',
 'YearBuilt',
 '2ndFlrSF',
 'GarageCars',
 'BsmtFinSF2',
 'BsmtFullBath',
 'LotFrontage',
 'GrLivArea',
 'GarageYrBlt',
 'Fireplaces',
 'TotRmsAbvGrd',
 'MiscVal',
 'TotalBsmtSF',
 'FullBath',
 'OverallQual',
 'OverallCond',
 '1stFlrSF',
 'WoodDeckSF',
 'BsmtHalfBath',
 'GarageArea',
 'LotArea',
 '3SsnPorch',
 'HalfBath',
 'YearRemodAdd',
 'BedroomAbvGr',
 'PoolArea',
 'BsmtFinSF1',
 'LowQualFinSF',
 'KitchenAbvGr',
 'MSSubClass',
 'BsmtUnfSF']

In [98]:
nominal = ['MSZoning', 'LandContour', 'Neighborhood', 'LotConfig', 'RoofMatl']
ordinal = list(set(cat_features) - set(nominal))

In [99]:
target = ['SalePrice']

**Use corresponding encoding mechanisms with the matching data**

In [100]:
for feature in ordinal:
  data[feature] = data[feature].astype('category').cat.codes

df_ordinal = data[ordinal]
df_ordinal

,GarageCond,Condition2,Condition1,Exterior2nd,ExterCond,Functional,PavedDrive,HeatingQC,MasVnrType,Heating,GarageQual,Foundation,BsmtExposure,BsmtFinType2,Electrical,SaleCondition,BldgType,CentralAir,GarageFinish,Utilities,LandSlope,BsmtQual,HouseStyle,Exterior1st,KitchenQual,LotShape,ExterQual,BsmtCond,SaleType,GarageType,Street,FireplaceQu,BsmtFinType1,RoofStyle
0,4,2,2,13,4,6,2,0,1,1,4,2,3,5,4,4,0,1,1,0,0,2,5,12,2,3,2,3,8,1,1,4,2,1
1,4,2,1,8,4,6,2,0,2,1,4,1,1,5,4,4,0,1,1,0,0,2,2,8,3,3,3,3,8,1,1,4,0,1
2,4,2,2,13,4,6,2,0,1,1,4,2,2,5,4,4,0,1,1,0,0,2,5,12,2,0,2,3,8,1,1,4,2,1
3,4,2,2,15,4,6,2,2,2,1,4,0,3,5,4,0,0,1,2,0,0,3,5,13,2,0,3,1,8,5,1,2,0,1
4,4,2,2,13,4,6,2,0,1,1,4,2,0,5,4,4,0,1,1,0,0,2,5,12,2,0,2,3,8,1,1,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,4,2,2,13,4,6,2,0,2,1,4,2,3,5,4,4,0,1,1,0,0,2,5,12,3,3,3,3,8,1,1,4,5,1
1456,4,2,2,10,4,2,2,4,3,1,4,1,3,4,4,4,0,1,2,0,0,2,2,9,3,3,3,3,8,1,1,4,0,1
1457,4,2,2,5,2,6,2,0,2,1,4,4,3,5,4,4,0,1,1,0,0,3,5,5,2,3,0,1,8,1,1,2,2,1
1458,4,2,2,8,4,6,2,2,2,1,4,1,2,4,0,4,0,1,2,0,0,3,2,8,2,3,3,3,8,1,1,2,2,3


In [101]:
df_nominal = pd.get_dummies(data[nominal])
df_nominal

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl
0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
1456,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
1457,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
1458,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0


In [102]:
df_numerical = data[numerical]

In [115]:
new_data = pd.concat([df_nominal, df_ordinal, df_numerical], axis=1)

In [114]:
new_data.head(3)

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,RoofMatl_ClyTile,...,FireplaceQu,BsmtFinType1,RoofStyle,EnclosedPorch,MasVnrArea,OpenPorchSF,ScreenPorch,MoSold,YrSold,SalePrice,YearBuilt,2ndFlrSF,GarageCars,BsmtFinSF2,BsmtFullBath,LotFrontage,GrLivArea,GarageYrBlt,Fireplaces,TotRmsAbvGrd,MiscVal,TotalBsmtSF,FullBath,OverallQual,OverallCond,1stFlrSF,WoodDeckSF,BsmtHalfBath,GarageArea,LotArea,3SsnPorch,HalfBath,YearRemodAdd,BedroomAbvGr,PoolArea,BsmtFinSF1,LowQualFinSF,KitchenAbvGr,MSSubClass,BsmtUnfSF
0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,4,2,1,0,196.0,61,0,2,2008,208500,2003,854,2,0,1,65.0,1710,2003.0,0,8,0,856,2,7,5,856,0,0,548,8450,0,1,2003,3,0,706,0,1,60,150
1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,...,4,0,1,0,0.0,0,0,5,2007,181500,1976,0,2,0,0,80.0,1262,1976.0,1,6,0,1262,2,6,8,1262,298,1,460,9600,0,0,1976,3,0,978,0,1,20,284
2,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,4,2,1,0,162.0,42,0,9,2008,223500,2001,866,2,0,1,68.0,1786,2001.0,1,6,0,920,2,7,5,920,0,0,608,11250,0,1,2002,3,0,486,0,1,60,434


In [117]:
x = new_data.drop('SalePrice', 1)
y = data[target]

**Standardizing the data (normalization)**

In [118]:
reg = LassoCV()
reg.fit(x,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [119]:
newer_data = PowerTransformer().fit_transform(new_data)

test_data = pd.DataFrame(newer_data,columns=new_data.columns)
test_data = pd.concat([test_data[ordinal],test_data[numerical],test_data[target]],axis=1)
test_data

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


,GarageCond,Condition2,Condition1,Exterior2nd,ExterCond,Functional,PavedDrive,HeatingQC,MasVnrType,Heating,GarageQual,Foundation,BsmtExposure,BsmtFinType2,Electrical,SaleCondition,BldgType,CentralAir,GarageFinish,Utilities,LandSlope,BsmtQual,HouseStyle,Exterior1st,KitchenQual,LotShape,ExterQual,BsmtCond,SaleType,GarageType,Street,FireplaceQu,BsmtFinType1,RoofStyle,EnclosedPorch,MasVnrArea,OpenPorchSF,ScreenPorch,MoSold,YrSold,SalePrice,YearBuilt,2ndFlrSF,GarageCars,BsmtFinSF2,BsmtFullBath,LotFrontage,GrLivArea,GarageYrBlt,Fireplaces,TotRmsAbvGrd,MiscVal,TotalBsmtSF,FullBath,OverallQual,OverallCond,1stFlrSF,WoodDeckSF,BsmtHalfBath,GarageArea,LotArea,3SsnPorch,HalfBath,YearRemodAdd,BedroomAbvGr,PoolArea,BsmtFinSF1,LowQualFinSF,KitchenAbvGr,MSSubClass,BsmtUnfSF,SalePrice
0,0.195977,-0.001447,0.053417,0.759104,0.372617,0.271156,0.299252,-0.956770,-1.235238,-0.098742,0.229416,0.836707,0.677875,0.360040,0.307296,0.032281,-0.44347,0.263813,-0.278812,-0.02618,-0.237571,-0.558697,1.018787,0.750372,-0.708836,0.752862,-1.148859,0.295157,0.388981,-0.703851,0.064238,1.129776,-0.331851,-0.399795,-0.407595,1.218044,0.841820,-0.293785,-1.694229,0.140053,0.571155,1.151867,1.170239,0.273835,-0.359384,1.175573,-0.156500,0.528531,1.118270,-1.027341,0.946380,-0.192177,-0.404029,0.797246,0.667499,-0.477387,-0.799624,-0.950337,-0.243940,0.383899,-0.141171,-0.129279,1.284128,0.908055,0.170752,-0.069409,0.797537,-0.134652,-0.172658,0.493460,-0.887576,0.571155
1,0.195977,-0.001447,-1.391852,-0.784551,0.372617,0.271156,0.299252,-0.956770,0.382090,-0.098742,0.229416,-0.542010,-1.404286,0.360040,0.307296,0.032281,-0.44347,0.263813,-0.278812,-0.02618,-0.237571,-0.558697,-0.453441,-0.647061,0.931869,0.752862,0.768841,0.295157,0.388981,-0.703851,0.064238,1.129776,-1.615804,-0.399795,-0.407595,-0.827466,-1.071307,-0.293785,-0.444891,-0.613715,0.227627,0.006695,-0.871454,0.273835,-0.359384,-0.838866,0.536617,-0.382755,-0.277361,0.784005,-0.234627,-0.192177,0.508177,0.797246,-0.042192,2.011352,0.429151,1.213714,4.099375,-0.004602,0.105609,-0.129279,-0.773692,-0.621670,0.170752,-0.069409,1.011471,-0.134652,-0.172658,-1.164269,-0.434388,0.227627
2,0.195977,-0.001447,0.053417,0.759104,0.372617,0.271156,0.299252,-0.956770,-1.235238,-0.098742,0.229416,0.836707,-0.636438,0.360040,0.307296,0.032281,-0.44347,0.263813,-0.278812,-0.02618,-0.237571,-0.558697,1.018787,0.750372,-0.708836,-1.371959,-1.148859,0.295157,0.388981,-0.703851,0.064238,1.129776,-0.331851,-0.399795,-0.407595,1.179948,0.668453,-0.293785,0.987490,0.140053,0.741869,1.055357,1.172896,0.273835,-0.359384,1.175573,-0.011160,0.659112,0.995940,0.784005,-0.234627,-0.192177,-0.253787,0.797246,0.667499,-0.477387,-0.568533,-0.950337,-0.243940,0.641990,0.413721,-0.129279,1.284128,0.835594,0.170752,-0.069409,0.571325,-0.134652,-0.172658,0.493460,-0.048083,0.741869
3,0.195977,-0.001447,0.053417,1.500145,0.372617,0.271156,0.299252,0.692110,0.382090,-0.098742,0.229416,-1.959038,0.677875,0.360040,0.307296,-2.599251,-0.44347,0.263813,1.016897,-0.02618,-0.237571,1.002516,1.018787,1.157540,-0.708836,-1.371959,0.768841,-3.389331,0.388981,1.388981,0.064238,-0.652025,-1.615804,-0.399795,2.456640,-0.827466,0.584540,-0.293785,-1.694229,-1.368874,-0.425386,-1.629540,1.146834,1.759713,-0.359384,1.175573,-0.407405,0.540797,0.818697,0.784005,0.389921,-0.192177,-0.644806,-1.025173,0.667499,-0.477387,-0.429428,-0.950337,-0.243940,0.786092,0.095490,-0.129279,-0.773692,-0.863478,0.170752,-0.069409,0.142272,-0.134652,-0.172658,0.698191,0.183337,-0.425386
4,0.195977,-0.001447,0.053417,0.759104,0.372617,0.271156,0.299252,-0.956770,-1.235238,-0.098742,0.229416,0.836707,-1.746093,0.360040,0.307296,0.032281,-0.44347,0.263813,-0.278812,-0.02618,-0.237571,-0.558697,1.018787,0.750372,-0.708836,-1.371959,-1.148859,0.295157,0.388981,-0.703851,0.064238,1.129776,-0.331851,-0.399795,-0.407595,1.324337,0.991895,-0.293785,1.963461,0.140053,1.015293,1.007861,1.209630,1.759713,-0.359384,1.175573,0.708754,1.282894,0.936036,0.784005,1.448615,-0.192177,0.

**Feature Selection with L1 embedding**

In [108]:
x.shape

(1460, 117)

In [ ]:
coef_col = pd.Series(reg.coef_,index = x.columns)

sorted_coef = coef_col.sort_values()
matplotlib.rcParams['figure.figsize']=(10.0,30.0)
sorted_coef.plot(kind="barh")

no_of_selected_features = sum(coef_col != 0)
no_of_rejected_features = sum(coef_col == 0)
total_features = no_of_selected_features + no_of_rejected_features
print("L1 selected only ",no_of_selected_features," features out of ",total_features," from the transformed dataset ")
plt.title("Feature Selection Using L1 Embedder")

In [149]:
coef_col

MSZoning_C (all)    -0.000000
MSZoning_FV          0.000000
MSZoning_RH         -0.000000
MSZoning_RL          0.000000
MSZoning_RM         -0.000000
                      ...    
BsmtFinSF1          12.170028
LowQualFinSF        -0.000000
KitchenAbvGr        -0.000000
MSSubClass          -0.000000
BsmtUnfSF            0.000000
Length: 117, dtype: float64

**Training a regression model using DFFA**

In [169]:
x = test_data.iloc[:, :-1].values
xy = test_data.iloc[:, :1].values

In [170]:
xTrain, xTest,yTrain, yTest = train_test_split(x, y, test_size=0.3, random_state=0)

In [172]:
xTrain.shape

(1022, 71)

In [175]:
yTrain.shape

(1022, 1)

In [173]:
xTest.shape

(438, 71)

In [174]:
yTest.shape

(438, 1)

In [194]:
model = keras.Sequential(
    [
        layers.Dense(4, activation="tanh"),
        layers.Dense(2, activation="tanh"),
        layers.Dense(1, activation='sigmoid')
    ]
)

x1 = tf.ones((1022, 71))
y1 = model(x1)

model.compile(loss = 'mse', optimizer = 'rmsprop', metrics = ['Precision', 'Recall'])

history = model.fit(xTrain, yTrain, epochs = 500, verbose = 0)

In [181]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 4)                 288       
_________________________________________________________________
dense_19 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 3         
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [195]:
historydf=pd.DataFrame(history.history, index=history.epoch)
historydf.head()

,loss,precision,recall
0,3.871096e+10,1.0,0.591977
1,3.871094e+10,1.0,0.673190
2,3.871094e+10,1.0,0.696673
3,3.871092e+10,1.0,0.743640
4,3.871091e+10,1.0,0.789628
